In [4]:
import TextualData.TextualData
import LSTM.lstm as lstm
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch
from torch.autograd import Variable
import utils
import spacy
import time
%load_ext snakeviz

In [42]:
hidden_size = 256
batch_size = 2
string_len = 2
valid_batches = 2
n_layers = 2
starting_lr = .025
lr_decay_factor = 2.
pos = 1
#Train the network to predict the charachter appearing after stride - Leave = 1
stride = 1
gpu = False

In [6]:
reload(TextualData.TextualData)
text = TextualData.TextualData.TextualData(path='data/full_shak_eng.txt', 
                                           lower=True, lines=100000, gpu=False)

In [7]:
print 'Computing POS...'
text.compute_pos()
print 'POS computed'
print 'alpha len', text.alpha_len, 'pos len', text.pos_len, 'product', text.alpha_len*text.pos_len

Computing POS...
Created POS data. Train_pos len: 60000 Valid_pos len: 20000
POS computed
alpha len 39 pos len 16 product 624


In [43]:
bc = text.get_batch(100,50,0,pos=True)
inputs = torch.stack([b[0] for b in bc])
labels = torch.stack([b[1] for b in bc])
if gpu is True:
    inputs=inputs.cuda()
    labels=labels.cuda()

In [44]:
if pos == 1:
    rnn = lstm.LSTMmodel(hidden_s=hidden_size, 
                         input_s=text.alpha_len*text.pos_len, n_layers=n_layers, gpu=gpu)
else:
    rnn = lstm.LSTMmodel(hidden_s=hidden_size, 
                         input_s=text.alpha_len, n_layers=n_layers, gpu=gpu)
if gpu is True:
    rnn=rnn.cuda()
rnn.optimizer = optim.Adam(rnn.parameters(), lr=starting_lr)

In [38]:
t=time.time()
np.mean([rnn.loss_func(rnn.forward(inputs[i]), labels[i]) for i in range(50)])
time.time()-t

0.865807056427002

In [46]:
t=time.time()
fw=rnn.forward(inputs)
print time.time()-t
t=time.time()
fw=[rnn.forward(i) for i in inputs]
time.time()-t

1.18831110001


1.6002740859985352

In [32]:
fw.size()

torch.Size([3, 624])

In [19]:
print_every = 5
update_every = 50
tot_loss=0
t = time.time()
valid_loss = np.mean([rnn.loss_func(rnn.forward(inp), tar) for inp, tar in
                      text.get_valid_batch(string_len,valid_batches,0,stride,pos)]).data[0]
print time.time()-t
print 'starting valid loss', valid_loss

0.00796794891357
starting valid loss 5.76587677002


In [25]:
%snakeviz 1+1

 
*** Profile stats marshalled to file u'/tmp/tmpNqXk2H'. 


In [22]:
inp, tar = text.get_valid_batch(string_len,valid_batches,0,stride,pos)
%snakeviz rnn.forward(inp[0])

 
*** Profile stats marshalled to file u'/tmp/tmpCJdLfs'. 


In [ ]:
for index in range(10):
    my_loss = rnn.train(text.get_batch(string_len,batch_size,index,stride,pos))
    print my_loss

In [13]:
print torch.cuda.is_available()

True


In [14]:
testvar = Variable(torch.cuda.LongTensor(np.zeros(10)))

In [17]:
type(testvar.cuda())

torch.autograd.variable.Variable

In [18]:
print testvar

Variable containing:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.cuda.LongTensor of size 10 (GPU 0)]



In [22]:
rnn.cuda()

LSTMmodel(
  (lstm): LSTM(256, 256, num_layers=2, dropout=0.25)
  (linear): Linear(in_features=256, out_features=848)
  (embedding): Linear(in_features=848, out_features=256)
  (logsoft): LogSoftmax()
  (loss_func): NLLLoss(
  )
)

In [16]:
mcp1=np.random.rand(150,45000)
mcp2=np.random.rand(45000,150)
mcp1gpu=torch.Tensor(mcp1).cuda()
mcp2gpu=torch.Tensor(mcp2).cuda()

In [19]:
t=time.time()
r1=[np.matmul(mcp1,mcp2) for i in range(100)]
print time.time()-t

4.2622449398


In [25]:
t=time.time()
r2=[torch.matmul(mcp1gpu,mcp2gpu) for i in range(100)]
print time.time()-t

0.00578308105469


In [33]:
r1[0][0]

array([11252.26541279, 11232.52284088, 11268.73060936, 11212.88691092,
       11264.94852029, 11245.24881578, 11291.23688647, 11204.89055259,
       11264.43886082, 11266.4704157 , 11264.8395667 , 11242.15466876,
       11275.41307562, 11283.38891701, 11227.61443004, 11252.20640174,
       11264.40516247, 11236.30388645, 11303.3395822 , 11235.141521  ,
       11246.36786597, 11236.13263528, 11162.67438639, 11221.64476667,
       11238.901956  , 11225.71865702, 11256.19390092, 11276.12626838,
       11294.25633129, 11259.66807267, 11256.3319513 , 11257.22308769,
       11261.12631686, 11258.53771383, 11285.778175  , 11227.73521397,
       11245.76426599, 11237.94470397, 11184.68507752, 11242.50919729,
       11230.03637027, 11335.32607909, 11302.09605179, 11261.48829391,
       11296.52778688, 11281.40835076, 11261.96775412, 11259.23744156,
       11231.03431951, 11294.75169432, 11214.27553918, 11291.45597562,
       11311.24953019, 11231.34006565, 11269.36284305, 11219.84541271,
      

In [34]:
r2[0][0]


 11252.2676
 11232.5254
 11268.7305
 11212.8867
 11264.9355
 11245.2500
 11291.2324
 11204.8828
 11264.4316
 11266.4727
 11264.8398
 11242.1602
 11275.4082
 11283.3887
 11227.6094
 11252.2100
 11264.4043
 11236.3027
 11303.3408
 11235.1426
 11246.3652
 11236.1299
 11162.6758
 11221.6523
 11238.8945
 11225.7227
 11256.1855
 11276.1328
 11294.2656
 11259.6670
 11256.3223
 11257.2227
 11261.1270
 11258.5410
 11285.7695
 11227.7275
 11245.7637
 11237.9502
 11184.6934
 11242.5127
 11230.0352
 11335.3223
 11302.0947
 11261.4834
 11296.5312
 11281.4023
 11261.9727
 11259.2344
 11231.0273
 11294.7451
 11214.2734
 11291.4541
 11311.2676
 11231.3379
 11269.3535
 11219.8477
 11253.8018
 11281.4453
 11287.1025
 11338.5156
 11228.2148
 11192.0957
 11236.6006
 11309.6719
 11256.4619
 11221.7227
 11273.4316
 11307.1943
 11288.3203
 11221.8047
 11250.3867
 11283.6348
 11210.8906
 11273.4258
 11272.1621
 11275.7617
 11252.3184
 11288.9531
 11297.9355
 11156.7168
 11323.1963
 11226.1768
 11248.5840
 11